In [19]:
import requests
from pprint import pprint
from sqlalchemy import create_engine
import pyodbc

In [11]:
# Create database engine
user = 'team_5_the_departed'
password = 'Spe@kFr1endAndEnter'
host = 'thedeparted.database.windows.net'
databasename = 'the_departed'
driver = 'ODBC+Driver+17+for+SQL+Server'
port = '1433'
engine = create_engine('mssql+pyodbc://{}:{}@{}:{}/{}?driver={}'.format(user, password, host, port, databasename, driver), echo=None)

In [20]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+host+';DATABASE='+databasename+';UID='+user+';PWD='+ password)
cursor = cnxn.cursor()

AttributeError: 'pyodbc.Cursor' object has no attribute 'fetch1'

In [12]:
from sqlalchemy.engine import URL
connection_string = "DRIVER={ODBC+Driver+17+for+SQL+Server};SERVER=tcp:thedeparted.database.windows.net;DATABASE=the_departed;UID=team_5_the_departed;PWD=Spe@kFr1endAndEnter"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine=create_engine(connection_url)

In [13]:
def get_weather(latitude, longitude):
    """Function to return a current weather response from openweather for a given set of co-ordinates"""
    weather_key = "7ac118753938be7d1540e9f996c5aab4"
    weather_by_coordinates = 'http://api.openweathermap.org/data/2.5/weather'
    r = requests.get(weather_by_coordinates,params={"APPID":weather_key, "lat":latitude, "lon":longitude })
    return r.json()

In [32]:
def store_weather(engine, connection):
    """
    Function to store weather dat in our database.
    Co-ordinates are hard-coded as the same co-ordinates as our historic weather data in the city centre
    Modified from weather scraper in previous project ()
    """
    position_lat = '53.349805'
    position_long = '-6.26031'
    # This calls our function to make the api call
    weather_json = get_weather(position_lat, position_long)
    
    weather_id=weather_json['weather'][0]['id']
    main=weather_json['weather'][0]['main']
    description=weather_json['weather'][0]['description']
    icon=weather_json['weather'][0]['icon']
    icon_url='http://openweathermap.org/img/wn/{}@2x.png'.format(icon)
    
    base=weather_json['base']
    temp=weather_json['main']['temp']
    feels_like=weather_json['main']['feels_like']
    temp_min=weather_json['main']['temp_min']
    temp_max=weather_json['main']['temp_max']
    pressure=weather_json['main']['pressure']
    humidity=weather_json['main']['humidity']
    visibility=weather_json['visibility']
    
    wind_speed=weather_json['wind']['speed']
    wind_degree=weather_json['wind']['deg']
    
    clouds_all=weather_json['clouds']['all']
    
    datetime=weather_json['dt']
    sys_type=weather_json['sys']['type']
    sys_country=weather_json['sys']['country']
    sys_id=weather_json['sys']['id']
    sys_sunrise=weather_json['sys']['sunrise']
    sys_sunset=weather_json['sys']['sunset']
    
    timezone=weather_json['timezone']
    id_var=weather_json['id']
    name=weather_json['name']
    cod=weather_json['cod']
    
    # Write an sql command to insert all this data into the current_weather table
    weather_insert='''INSERT INTO current_weather
    
                                (    position_long
                                    ,position_lat
                                    ,weather_id
                                    ,main
                                    ,description
                                    ,icon
                                    ,icon_url
                                    ,base
                                    ,temp
                                    ,feels_like
                                    ,temp_min
                                    ,temp_max
                                    ,pressure
                                    ,humidity
                                    ,visibility
                                    ,wind_speed
                                    ,wind_degree
                                    ,clouds_all
                                    ,datetime
                                    ,sys_type
                                    ,sys_id
                                    ,sys_country
                                    ,sys_sunrise
                                    ,sys_sunset
                                    ,timezone
                                    ,id
                                    ,name
                                    ,cod)
                                  
                            VALUES
                                (%s
                                  ,%s
                                  ,%s
                                  
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  
                                  ,%s
                                  ,%s
                                  
                                  ,%s
                                  
                                  ,%s
                                  ,%s
                                  ,%s
                                  ,%s
                                  
                                  ,%s
                                  ,%s
                                  
                                  ,%s
                                  ,%s 
                                  ,%s
                                  ,%s
                                  ,%s)
                                    '''
    weather_values=(position_long
                    ,position_lat
                    ,weather_id
                    ,main
                    ,description
                    ,icon
                    ,icon_url
                    ,base
                    ,temp
                    ,feels_like
                    ,temp_min
                    ,temp_max
                    ,pressure
                    ,humidity
                    ,visibility
                    ,wind_speed
                    ,wind_degree
                    ,clouds_all
                    ,datetime
                    ,sys_type
                    ,sys_id
                    ,sys_country
                    ,sys_sunrise
                    ,sys_sunset
                    ,timezone
                    ,id_var
                    ,name
                    ,cod)
    engine.execute(weather_insert,weather_values)
    connection.commit()

In [15]:
db_schema = {
    'current_weather':{
        'entry_id': 'INTEGER'
        ,'dt': 'DATETIME'
        ,'weather_id':'INTEGER'
        ,'weather_main': 'VARCHAR(256)'
        ,'coord_lon': 'REAL'
        ,'coord_lat': 'REAL'
        ,'main':'VARCHAR(256)'
        ,'weather_description':'VARCHAR(500)'
        ,'weather_icon':'VARCHAR(20)'
        ,'base':'varchar(256)'
        ,'main_temp':'REAL'
        ,'main_feels_like':'REAL'
        ,'main_temp_min':'REAL'
        ,'main_temp_max':'REAL'
        ,'main_pressure':'INT'
        ,'main_humidity':'INT'
        ,'visibility':'INT'
        ,'wind_speed':'REAL'
        ,'wind_deg':'INT'
        ,'clouds_all':'INT'
        ,'sys_type':'INT'
        ,'sys_id':'INT'
        ,'sys_country':'VARCHAR(10)'
        ,'sys_sunrise':'BIGINT'
        ,'sys_sunset':'BIGINT'
        ,'timezone':'INT'
        ,'id':'BIGINT'
        ,'name':'VARCHAR(256)'
        ,'cod':'INT'
        }
}

In [26]:
def create_db_tables(engine, connection):
    """Function to create tables in our database if they do not already exist"""
    for table, columns in db_schema.items():
        column_count = 0
        #sql = """CREATE TABLE IF NOT EXISTS {} (\n""".format(table)
        sql = """if not exists (select * from sysobjects where name='current_weather' and xtype='U')
        CREATE TABLE current_weather (\n"""
        
        insert_row = ''
        
        for column_heading, data_type in columns.items():
            if column_count > 0:
                insert_row += ",\n\t {} {}".format(column_heading, data_type)
            else:
                # Create an auto incremented id column as the very first column.
                insert_row += "\t{} {} NOT NULL IDENTITY PRIMARY KEY".format(column_heading, data_type)
                column_count += 1
        sql += "{})".format(insert_row)
        #conn = engine.connect()
        print(sql)
        engine.execute(sql)
        #engine.fetchone()
        connection.commit
       # engine.execute(sql)

In [27]:
create_db_tables(cursor, cnxn)

if not exists (select * from sysobjects where name='current_weather' and xtype='U')
        CREATE TABLE current_weather (
	entry_id INTEGER NOT NULL IDENTITY PRIMARY KEY,
	 dt DATETIME,
	 weather_id INTEGER,
	 weather_main VARCHAR(256),
	 coord_lon REAL,
	 coord_lat REAL,
	 main VARCHAR(256),
	 weather_description VARCHAR(500),
	 weather_icon VARCHAR(20),
	 base varchar(256),
	 main_temp REAL,
	 main_feels_like REAL,
	 main_temp_min REAL,
	 main_temp_max REAL,
	 main_pressure INT,
	 main_humidity INT,
	 visibility INT,
	 wind_speed REAL,
	 wind_deg INT,
	 clouds_all INT,
	 sys_type INT,
	 sys_id INT,
	 sys_country VARCHAR(10),
	 sys_sunrise BIGINT,
	 sys_sunset BIGINT,
	 timezone INT,
	 id BIGINT,
	 name VARCHAR(256),
	 cod INT)


In [33]:
store_weather(cursor, cnxn)

ProgrammingError: ('The SQL contains 0 parameter markers, but 28 parameters were supplied', 'HY000')

In [ ]:
test = """
SELECT current_weather
FROM the_departed.current_weather
"""

In [ ]:
engine.execute(test)

In [ ]:
# Get an API response for the same co-ordinates as our historic weather data
DublinWeather = get_weather('53.349805', '-6.26031')
pprint(DublinWeather)